# Spark DataFrame Basics

https://spark.apache.org/docs/latest/sql-getting-started.html

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Imports

In [2]:
from pyspark.sql import SparkSession

### Start Spark Session

In [4]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

### Reading Data

In [5]:
people_json = 'people.json'

In [6]:
df = spark.read.json(people_json)

In [7]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [8]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [9]:
df.columns

['age', 'name']

In [10]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



### DF Schema

In [11]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [12]:
data_schema = [StructField('age', IntegerType(), True),
               StructField('name', StringType(), True)]

In [13]:
final_struct = StructType(fields=data_schema)

In [14]:
df = spark.read.json(people_json, schema=final_struct)

In [15]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [16]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Manipulating Columns

In [17]:
df.withColumn('double_age', df['age']*2)

DataFrame[age: int, name: string, double_age: int]

In [18]:
df.withColumnRenamed('age', 'my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



### Using SQL in Spark

In [19]:
df.createOrReplaceTempView('people')

In [20]:
results = spark.sql('SELECT * FROM people')

In [21]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [22]:
new_results = spark.sql("SELECT * FROM people WHERE age=30")

In [23]:
new_results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



In [24]:
spark.stop()

# Data Manipulation

In [25]:
from pyspark.sql import SparkSession

In [28]:
spark = SparkSession.builder.appName('ops').getOrCreate()

In [29]:
apple_stock = 'appl_stock.csv'

In [30]:
df = spark.read.csv(apple_stock, header=True, inferSchema=True)

In [31]:
df.show(10)

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [32]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



### Filter

In [33]:
df.filter('Close < 500').show(10)

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [34]:
df.filter('Close < 500').select('Open').show(10)

+------------------+
|              Open|
+------------------+
|        213.429998|
|        214.599998|
|        214.379993|
|            211.75|
|        210.299994|
|212.79999700000002|
|209.18999499999998|
|        207.870005|
|210.11000299999998|
|210.92999500000002|
+------------------+
only showing top 10 rows



In [35]:
df.filter(df['Close'] > 500).show(10)

+----------+------------------+------------------+------------------+------------------+---------+-----------------+
|      Date|              Open|              High|               Low|             Close|   Volume|        Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+-----------------+
|2012-02-13|        499.529991|503.83000899999996|497.08998899999995|502.60002099999997|129304000|        65.116633|
|2012-02-14|        504.659988|         509.56002|        502.000008|        509.459991|115099600|        66.005408|
|2012-02-16|        491.500008|        504.890007|         486.62999|502.20999900000004|236138000|        65.066102|
|2012-02-17|        503.109993|507.77002000000005|        500.299995|         502.12001|133951300|        65.054443|
|2012-02-21|506.88001299999996|        514.850021|504.12000300000005|        514.850021|151398800|        66.703738|
|2012-02-22|        513.079994|        515.489983|509.0700230000

In [36]:
df.filter(df['Close'] > 500).select('Volume').show(10)

+---------+
|   Volume|
+---------+
|129304000|
|115099600|
|236138000|
|133951300|
|151398800|
|120825600|
|142006900|
|103768000|
|136895500|
|150096800|
+---------+
only showing top 10 rows



In [37]:
df.filter((df['Close'] < 200) & (df['Open']>200)).show()

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



In [38]:
df.filter((df['Close'] < 200) & ~(df['Open']>200)).show(10)

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-02-01|192.36999699999998|             196.0|191.29999899999999|        194.729998|187469100|         25.229131|
|2010-02-02|        195.909998|        196.319994|193.37999299999998|        195.859997|174585600|25.375532999999997|
|2010-02-03|        195.169994|        200.200003|        194.420004|        199.229994|153832000|25.812148999999998|
|2010-02-04|        196.730003|        198.370001|        191.570005|        192.050003|189413000|         24.881912|
|2010-02-05|192.63000300000002|             196.0|        190.850002|        195.460001|212576700|25.323710000000002|
|2010-02-08|        195.690006|197.88000300000002|      

In [39]:
df.filter(df['Low'] == 486.62999).show()

+----------+----------+----------+---------+------------------+---------+---------+
|      Date|      Open|      High|      Low|             Close|   Volume|Adj Close|
+----------+----------+----------+---------+------------------+---------+---------+
|2012-02-16|491.500008|504.890007|486.62999|502.20999900000004|236138000|65.066102|
+----------+----------+----------+---------+------------------+---------+---------+



In [40]:
spark.stop()

# Groupby and Aggregates

In [41]:
from pyspark.sql import SparkSession

In [42]:
sales_info = 'sales_info.csv'

In [44]:
spark = SparkSession.builder.appName('agga').getOrCreate()

In [45]:
df = spark.read.csv(sales_info, header=True, inferSchema=True)

In [46]:
df.count()

12

In [47]:
df.show(10)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
+-------+-------+-----+
only showing top 10 rows



In [48]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



### Groupby apply

In [49]:
df.groupBy('Company').mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [50]:
df.groupBy('Company').max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [51]:
df.groupBy('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [52]:
df.agg({'Sales':'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [53]:
df.agg({'Sales':'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [54]:
group_data = df.groupBy('Company')

In [55]:
group_data.agg({'Sales':'max', 'Person':'min'}).show()

+-------+----------+-----------+
|Company|max(Sales)|min(Person)|
+-------+----------+-----------+
|   APPL|     750.0|      Chris|
|   GOOG|     340.0|    Charlie|
|     FB|     870.0|       Carl|
|   MSFT|     600.0|        Amy|
+-------+----------+-----------+



### Other Functions

In [56]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [57]:
df.select(countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [58]:
df.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



### Format column and renaming

In [59]:
df.select(avg('Sales').alias('Average Sales')).show()

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [60]:
from pyspark.sql.functions import format_number

In [61]:
sales_std = df.select(stddev('Sales'))

In [62]:
sales_std.select(format_number('stddev_samp(Sales)', 2).alias('std of sales')).show()

+------------+
|std of sales|
+------------+
|      250.09|
+------------+



In [63]:
df.orderBy('Sales', ).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [64]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



In [65]:
spark.stop()

# Missing Data

In [66]:
from pyspark.sql import SparkSession

In [68]:
spark = SparkSession.builder.appName('Miss').getOrCreate()

In [69]:
contains_null = 'ContainsNull.csv'

In [70]:
df = spark.read.csv(contains_null, header=True, inferSchema=True)

In [71]:
df.show(10)

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Dropping Null Values

In [72]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [73]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [74]:
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [75]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [76]:
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Fill missing values

In [77]:
df.na.fill('FILL VALUE').show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|FILL VALUE| null|
|emp3|FILL VALUE|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [78]:
df.na.fill(9999).show()

+----+-----+------+
|  Id| Name| Sales|
+----+-----+------+
|emp1| John|9999.0|
|emp2| null|9999.0|
|emp3| null| 345.0|
|emp4|Cindy| 456.0|
+----+-----+------+



In [79]:
df.na.fill('FILL VALUE', subset=['Name']).show()

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|FILL VALUE| null|
|emp3|FILL VALUE|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [80]:
from pyspark.sql.functions import mean

In [81]:
mean_val = df.select(mean(df['Sales'])).collect()

In [82]:
mean_sales = mean_val[0][0]

In [83]:
df.na.fill(mean_sales, subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [84]:
spark.stop()

# Dates and Timestamps

In [85]:
from pyspark.sql import SparkSession

In [87]:
spark = SparkSession.builder.appName('dates').getOrCreate()

In [88]:
df = spark.read.csv(apple_stock, header=True, inferSchema=True)

In [89]:
df.select(['Date', 'Open']).show(5)

+----------+----------+
|      Date|      Open|
+----------+----------+
|2010-01-04|213.429998|
|2010-01-05|214.599998|
|2010-01-06|214.379993|
|2010-01-07|    211.75|
|2010-01-08|210.299994|
+----------+----------+
only showing top 5 rows



### Extract Date Information

In [90]:
from pyspark.sql.functions import (dayofmonth, hour, dayofyear, month, year, weekofyear, format_number, date_format)

In [91]:
df.select(dayofmonth(df['Date'])).show(5)

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
+----------------+
only showing top 5 rows



In [92]:
df.select(hour(df['Date'])).show(5)

+----------+
|hour(Date)|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
+----------+
only showing top 5 rows



In [93]:
df.select(month(df['Date'])).show(5)

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
|          1|
|          1|
|          1|
+-----------+
only showing top 5 rows



In [94]:
df.select(year(df['Date'])).show(5)

+----------+
|year(Date)|
+----------+
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
+----------+
only showing top 5 rows



In [95]:
new_df = df.withColumn("Year", year(df['Date']))

In [96]:
new_df.show(5)

+----------+----------+----------+------------------+------------------+---------+------------------+----+
|      Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|Year|
+----------+----------+----------+------------------+------------------+---------+------------------+----+
|2010-01-04|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|2010|
|2010-01-05|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|2010|
|2010-01-06|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|2010|
|2010-01-07|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|2010|
|2010-01-08|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|2010|
+----------+----------+----------+------------------+------------------+---------+------------------+----+
only showing top 5 rows



In [97]:
result =  new_df.groupby('Year').mean().select(['Year','avg(Close)'])

In [98]:
result_new = result.withColumnRenamed('avg(Close)','Average Closing Price')

In [99]:
result_new.show()

+----+---------------------+
|Year|Average Closing Price|
+----+---------------------+
|2015|   120.03999980555547|
|2013|    472.6348802857143|
|2014|    295.4023416507935|
|2012|    576.0497195640002|
|2016|   104.60400786904763|
|2010|    259.8424600000002|
|2011|   364.00432532142867|
+----+---------------------+



In [100]:
result_new.select(['Year',format_number('Average Closing Price',2).alias('Avg Closing Price')]).show()

+----+-----------------+
|Year|Avg Closing Price|
+----+-----------------+
|2015|           120.04|
|2013|           472.63|
|2014|           295.40|
|2012|           576.05|
|2016|           104.60|
|2010|           259.84|
|2011|           364.00|
+----+-----------------+



In [101]:
spark.stop()